<html><h1><center>Applying LSTM Models on Raw Data</center></h1></html>

In [35]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2


In [36]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [53]:
import matplotlib.pyplot as plt
import seaborn as sns

# function to print the confusion matrix

def new_confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])
    plt.show()  

## Loading Data

In [38]:
# Data directory
DATADIR = '/kaggle/input/d/quanganh2001/uci-har-dataset/UCI HAR Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration


SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [39]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/kaggle/input/d/quanganh2001/uci-har-dataset/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [40]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/kaggle/input/d/quanganh2001/uci-har-dataset/UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [41]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [42]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [43]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [44]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [45]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [46]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


## 1. Defining the Architecture of 1-Layer of LSTM

In [47]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

2023-02-20 09:38:59.578420: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64::/opt/conda/lib
2023-02-20 09:38:59.578483: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [49]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test),epochs=epochs)

Epoch 1/30
460/460 [==============================] - 30s 58ms/step - loss: 1.1758 - accuracy: 0.5054 - val_loss: 0.9540 - val_accuracy: 0.6128
Epoch 2/30
460/460 [==============================] - 26s 56ms/step - loss: 0.8117 - accuracy: 0.6500 - val_loss: 0.7881 - val_accuracy: 0.6888
Epoch 3/30
460/460 [==============================] - 27s 58ms/step - loss: 0.6823 - accuracy: 0.7178 - val_loss: 0.7053 - val_accuracy: 0.7251
Epoch 4/30
460/460 [==============================] - 26s 57ms/step - loss: 0.5750 - accuracy: 0.7911 - val_loss: 0.5702 - val_accuracy: 0.7995
Epoch 5/30
460/460 [==============================] - 27s 59ms/step - loss: 0.4317 - accuracy: 0.8670 - val_loss: 0.4970 - val_accuracy: 0.8409
Epoch 6/30
460/460 [==============================] - 26s 56ms/step - loss: 0.3548 - accuracy: 0.8898 - val_loss: 0.4422 - val_accuracy: 0.8578
Epoch 7/30
460/460 [==============================] - 27s 59ms/step - loss: 0.3113 - accuracy: 0.9006 - val_loss: 0.5196 - val_accuracy:

In [54]:
# Confusion Matrix
new_confusion_matrix(Y_test, model.predict(X_test))

93/93 [==============================] - 1s 16ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,27,0,0,0
SITTING,0,385,103,1,0,2
STANDING,0,77,453,2,0,0
WALKING,0,0,0,464,14,18
WALKING_DOWNSTAIRS,0,1,1,3,407,8
WALKING_UPSTAIRS,0,0,0,14,19,438


In [55]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 2s 17ms/step - loss: 0.5395 - accuracy: 0.9016

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.5395119190216064, 0.9015948176383972]


With a simple 1 layer architecture we got 90.15% accuracy and a loss of 0.53

## 2. Defining the Architecture of 2-Layer of LSTM with more hyperparameter tunning

### 2.1 First Model for 2-Layer of LSTM with more hyperparameter tunning

In [56]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(Y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)

In [57]:
# Model execution
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True,bias_regularizer=reg ))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128, 48)           11136     
                                                                 
 batch_normalization (BatchN  (None, 128, 48)          192       
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 128, 48)           0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                10368     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                      

In [58]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [59]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 65s 131ms/step - loss: 1.5544 - accuracy: 0.6757 - val_loss: 0.9445 - val_accuracy: 0.8297
Epoch 2/30
460/460 [==============================] - 60s 130ms/step - loss: 0.7333 - accuracy: 0.8633 - val_loss: 0.6206 - val_accuracy: 0.8225
Epoch 3/30
460/460 [==============================] - 58s 125ms/step - loss: 0.4229 - accuracy: 0.9029 - val_loss: 0.2909 - val_accuracy: 0.9192
Epoch 4/30
460/460 [==============================] - 58s 126ms/step - loss: 0.2433 - accuracy: 0.9320 - val_loss: 0.2723 - val_accuracy: 0.9152
Epoch 5/30
460/460 [==============================] - 60s 131ms/step - loss: 0.2188 - accuracy: 0.9283 - val_loss: 0.3772 - val_accuracy: 0.8819
Epoch 6/30
460/460 [==============================] - 60s 130ms/step - loss: 0.2214 - accuracy: 0.9251 - val_loss: 0.2298 - val_accuracy: 0.9250
Epoch 7/30
460/460 [==============================] - 59s 129ms/step - loss: 0.2151 - accuracy: 0.9287 - val_loss: 0.2605 - val_ac

In [60]:
# Confusion Matrix
new_confusion_matrix(Y_test, model.predict(X_test))

93/93 [==============================] - 4s 35ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,1,364,123,0,0,3
STANDING,0,52,480,0,0,0
WALKING,0,0,6,490,0,0
WALKING_DOWNSTAIRS,0,0,0,4,416,0
WALKING_UPSTAIRS,0,0,0,23,3,445


In [61]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 3s 37ms/step - loss: 0.2387 - accuracy: 0.9270

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.23867270350456238, 0.9270444512367249]


### 2.2 Second Model for 2-Layer of LSTM with more hyperparameter tunning

In [62]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 128, 64)           18944     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 64)          256       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_4 (LSTM)               (None, 48)                21696     
                                                                 
 dropout_4 (Dropout)         (None, 48)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 294       
                                      

In [63]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 69s 140ms/step - loss: 1.6160 - accuracy: 0.7439 - val_loss: 1.3473 - val_accuracy: 0.7343
Epoch 2/30
460/460 [==============================] - 63s 138ms/step - loss: 0.7320 - accuracy: 0.8931 - val_loss: 2.2387 - val_accuracy: 0.5389
Epoch 3/30
460/460 [==============================] - 64s 138ms/step - loss: 0.3674 - accuracy: 0.9138 - val_loss: 0.3720 - val_accuracy: 0.8901
Epoch 4/30
460/460 [==============================] - 63s 136ms/step - loss: 0.1959 - accuracy: 0.9346 - val_loss: 0.4738 - val_accuracy: 0.8704
Epoch 5/30
460/460 [==============================] - 64s 138ms/step - loss: 0.1969 - accuracy: 0.9327 - val_loss: 0.5283 - val_accuracy: 0.8419
Epoch 6/30
460/460 [==============================] - 63s 136ms/step - loss: 0.1631 - accuracy: 0.9373 - val_loss: 0.2492 - val_accuracy: 0.9165
Epoch 7/30
460/460 [==============================] - 64s 140ms/step - loss: 0.1573 - accuracy: 0.9421 - val_loss: 0.2044 - val_ac

In [65]:
# Confusion Matrix
new_confusion_matrix(Y_test, model.predict(X_test))

93/93 [==============================] - 4s 38ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,5,384,101,0,0,1
STANDING,0,97,432,3,0,0
WALKING,0,3,2,476,14,1
WALKING_DOWNSTAIRS,0,0,0,2,410,8
WALKING_UPSTAIRS,0,1,0,14,0,456


In [66]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 4s 38ms/step - loss: 0.2339 - accuracy: 0.9145

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.23387710750102997, 0.9144893288612366]
